<h1 style="text-align: center; font-size=58px;">Your First Delete</h1>

As usual, we'll import MongoClient and set up our connection uri.

In [ ]:
from pymongo import MongoClient

And then intialization our connection and get back a MongoClient object.

In [ ]:
client = MongoClient()

Since we're learning about deletes in this lesson and don't want to work with any of our production data, we'll define a new database and collection name to work with.

In [ ]:
lessons = client.lessons
deletes = lessons.deletes

Now that we have a collection object named **deletes** with no data in it, let's insert some data.

We'll insert 100 documents with an **_id** that ranges from 0 to 99, and a field called **random_bool** that will randomly be true or false. We'll run an assertion stating that we expect 100 ObjectIds to have been inserted. If this isn't true we'll see an error.

We've added the drop method at the beginning of this code cell to ensure repeatability in case we want to run through this lesson again.

In [ ]:
import random
random.seed(42)
deletes.drop()
imr = deletes.insert_many([{'_id': val, 'random_bool': random.choice([True, False])} for val in range(100)])
assert len(imr.inserted_ids) == 100

Ok, let's grab the first 3 documents to get a sense for what they look like.

In [ ]:
list(deletes.find().limit(3))

Ok, we're convinced that we have a fairly random **random_bool** field and an **_id** with values between 0 and 99.

We've learned how to create, read, and update documents. Now to delete.

**pymongo** offers two idiomatic delete methods, **delete_one** and **delete_many**. Let's look at them both to get a sense for how they work.

<h1 style="text-align: center; font-size=58px;"><pre>delete_one</pre></h1>

`delete_one` is a lot like `find_one`. It takes a predicate to match the document you want to delete, finds the document, and deletes it. If multiple documents match the predicate, `delete_one` will only delete the first document matched.

Let's use `delete_one` to delete the first document where **random_bool** is True. Based on what I said, we should be left with 99 documents in the collection.

We'll assign the DeleteResult object to the variable **dr** so we can print out the **deleted_count** property which tells us how many documents were deleted.

In [ ]:
dr = deletes.delete_one({'random_bool': True})
dr.deleted_count

`delete_one` can be thought of like a precision scalpel. If we know some value or values that uniquely identify a document, we're guaranteed to only delete that document.

We know the **_id** must be unique, so let's delete the document with **'_id': 99**

First we'll find the document to prove it exists, then delete it, then try to find it again. We should get None back for the second find.

In [ ]:
deletes.find_one({'_id': 99})

In [ ]:
deletes.delete_one({'_id': 99})

In [ ]:
deletes.find_one({'_id': 99})

<h1 style="text-align: center; font-size=58px;"><pre>delete_many</pre></h1>

Unlike `delete_one`, `delete_many` deletes all documents that match the supplied predicate. Because of this behavior, `delete_many` is a little more "dangerous".

Let's first get a count of how many documents now have False and True for their **random_bool** value. Then, we'll use `delete_many` to delete **all** documents where **random_bool** is False.

In [ ]:
len(list(deletes.find({'random_bool': False})))

In [ ]:
len(list(deletes.find({'random_bool': True})))

44 documents have a **random_bool** value of False. Our deleted count should be 44, and a count on the collection should yield 54.

In [ ]:
dr = deletes.delete_many({'random_bool': False})
dr.deleted_count

In [ ]:
len(list(deletes.find({'random_bool': True})))

## Summary

And that covers the basics of deleting documents with pymongo. Remember

* `delete_one` will delete the first document that matches the supplied predicate.
* `delete_many` will delete all documents matching the supplied predicate.
* The number of documents deleted can be accessed via the **deleted_count** property on the `DeleteResult` object returned from a delete operation.